**Objectif : Le présent notebook servira à démontrer comment tester l'API que nous avons déployée**

## <font color='Blue'> **Partie 1 : Création de l'Application Streamlit en Local**

### <font color='RoyalBlue'> 1. Import de librairies necessaires

In [67]:
import os
import joblib
import streamlit as st

### <font color='RoyalBlue'> 2. Écrire le Code pour l'Application Streamlit en Local

**Crée un fichier Python, par exemple app_local.py, avec le code suivant :**

In [71]:
# Titre de l'application
st.title("Prédiction avec votre modèle (Test avec SK_ID_CURR spécifique)")

# Définir le chemin absolu du modèle
model_path = os.path.abspath(os.path.join("saved_model", "best_lgbmb_model.joblib"))
st.write(f"Chargement du modèle depuis {model_path}")

# Charger le modèle
try:
    model = joblib.load(model_path)
    st.write("Modèle chargé avec succès")
except FileNotFoundError:
    st.error(f"Le fichier {model_path} est introuvable. Veuillez vérifier le chemin du fichier.")

# Interface utilisateur pour entrer des données
sk_id_curr = st.text_input("Entrez SK_ID_CURR", "100004")  # Par défaut à 100004

if st.button("Prédire"):
    # Charger vos données et faire une prédiction
    df_path = os.path.abspath(os.path.join("saved_model", "df_train_smote_corrected_100rows_with_id.joblib"))
    st.write(f"Chargement du DataFrame depuis {df_path}")
    try:
        df = joblib.load(df_path)
        st.write("DataFrame chargé avec succès")
        
        if int(sk_id_curr) in df['SK_ID_CURR'].values:
            sample = df[df['SK_ID_CURR'] == int(sk_id_curr)]

            # Liste des colonnes attendues par le modèle
            model_columns = model.booster_.feature_name()

            # Afficher un aperçu des premières colonnes
            st.write("Colonnes du DataFrame :")
            st.dataframe(sample.iloc[:, :10])  # Affiche les 10 premières colonnes

            # Obtenir la probabilité prédite
            prediction_proba = model.predict_proba(sample[model_columns])
            probability = prediction_proba[0][1]  # Probabilité de la classe positive
            st.write(f"Probabilité prédite : {probability * 100:.2f}%")
        else:
            st.write(f"L'ID {sk_id_curr} n'existe pas dans le DataFrame.")
    
    except FileNotFoundError:
        st.error(f"Le fichier {df_path} est introuvable. Veuillez vérifier le chemin du fichier.")

2024-09-01 13:51:28.931 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-01 13:51:28.933 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-01 13:51:28.937 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-01 13:51:28.938 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-01 13:51:28.940 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-01 13:51:28.941 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-01 13:51:28.946 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-01 13:51:28.948 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

### <font color='RoyalBlue'> 3. Exécuter l'Application en Local

**streamlit run app_local.py**
*****
***Essaie de saisir 100004 ou 100002 comme SK_ID_CURR***

### <font color='RoyalBlue'> 4. Test API en local avec API Flask

In [109]:
import requests

# Définir l'URL de l'API
url = "http://192.168.0.103:5000/predict"

# Paramètres à envoyer dans l'URL
params = {
    "SK_ID_CURR": "100004" 
}

# Faire une requête GET à l'API
response = requests.get(url=url, params=params)

# Vérifier le statut de la réponse
print(f"Statut de la réponse : {response.status_code}")

# Afficher la réponse de l'API
if response.status_code == 200:
    print(response.json())
else:
    print(f"Erreur : {response.text}")

Statut de la réponse : 200
{'feature_names': ['Column_0', 'Column_1', 'Column_2', 'Column_3', 'Column_4', 'Column_5', 'Column_6', 'Column_7', 'Column_8', 'Column_9'], 'feature_values': [-0.1954256211947472, 0.717813511857255, -0.71740951853553, -0.6642391539220147, -0.5750549265883934, 0.2969115167370191, -0.2311222202629274, -0.371691309326604, -0.811258969882764, 0.1767918301208186], 'probability': 11.05, 'shap_values': [0.003955517956788203, -0.4085871507065908, -0.06607947025075532, -0.09171127557901478, -0.06122439880549777, -0.0072747781791684665, 0.03228563325666001, 0.013843880890944094, -0.027505460487406035, 0.06708550752839927]}


## <font color='Blue'>**Partie 2 : Création de l'Application Streamlit pour Déploiement en Ligne**

### <font color='RoyalBlue'> 1. Adapter le Code pour le Déploiement en Ligne

***D'abord nous créons un fichier Python, par exemple app_online.py, qui sera adapté pour fonctionner sur une plateforme comme Streamlit Cloud,ensuite nous Remplacons les chemins locaux par des chemins relatifs ou absolus en fonction de l'organisation du projet.***

In [78]:
# Titre de l'application
st.title("Prédiction avec votre modèle (En ligne)")

# Définir le chemin du modèle pour usage en ligne
model_path = os.path.join("saved_model", "best_lgbmb_model.joblib")
st.write(f"Chargement du modèle depuis {model_path}")

# Vérification de l'existence du fichier de modèle
if not os.path.exists(model_path):
    st.error(f"Le fichier modèle {model_path} n'existe pas.")
else:
    # Charger le modèle
    model = joblib.load(model_path)
    st.write("Modèle chargé avec succès")

    # Interface utilisateur pour entrer des données
    sk_id_curr = st.text_input("Entrez SK_ID_CURR")

    if st.button("Prédire"):
        # Charger nos données et faire une prédiction
        df_path = os.path.join("saved_model", "df_train_smote_corrected_100rows_with_id.joblib")
        st.write(f"Chargement du DataFrame depuis {df_path}")

        # Vérification de l'existence du fichier DataFrame
        if not os.path.exists(df_path):
            st.error(f"Le fichier DataFrame {df_path} n'existe pas.")
        else:
            df = joblib.load(df_path)
            st.write("DataFrame chargé avec succès")

            # Trouver l'échantillon correspondant au SK_ID_CURR
            try:
                sample = df[df['SK_ID_CURR'] == int(sk_id_curr)]
            except ValueError:
                st.error("Veuillez entrer un SK_ID_CURR valide.")
                sample = None

            if sample is None or sample.empty:
                st.write(f"Aucun échantillon trouvé pour SK_ID_CURR: {sk_id_curr}")
            else:
                # Liste des colonnes attendues par le modèle
                model_columns = model.booster_.feature_name()

                # Vérification que toutes les colonnes nécessaires sont présentes dans le DataFrame
                missing_columns = [col for col in model_columns if col not in sample.columns]
                if missing_columns:
                    st.error(f"Les colonnes suivantes sont manquantes dans le DataFrame: {missing_columns}")
                else:
                    # Obtenir la probabilité prédite
                    prediction_proba = model.predict_proba(sample[model_columns])
                    probability = prediction_proba[0][1]  # Probabilité de la classe positive
                    st.write(f"Probabilité prédite : {probability * 100:.2f}%")

2024-09-01 13:51:32.603 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-01 13:51:32.604 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-01 13:51:32.605 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-01 13:51:32.606 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-01 13:51:32.607 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-01 13:51:32.608 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-01 13:51:32.610 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-01 13:51:32.611 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

### <font color='RoyalBlue'> 2. Déployer l'Application en Ligne

- **Étape 1 : Pousser notre  code sur un dépôt GitHub public ou privé.**

- **Étape 2 : Sur Streamlit Cloud, connectonssur notre compte GitHub.**

- **Étape 3 : Choisissons notre dépôt et le fichier app_online.py.**

- **Étape 4 : Déploieons l'application en suivant les instructions.**

### <font color='RoyalBlue'> 3. Accéder à l'Application en Ligne

***Après déploiement, Streamlit nous fournira un lien vers l'application en ligne que nous pouvons le partager et l'utiliser pour la démonstration.***

<font color='Blue'>**LIEN**
- **[Accéder à l'application Streamlit](https://samiram-ux.streamlit.app)**
******

## <font color='Blue'>**Partie 3 : Tests et Validation**

In [105]:
import requests

url = "https://samiram-ux.streamlit.app"
params = {
    "sk_id_curr": 100004
}

response = requests.get(url, params=params)
print("Status Code:", response.status_code)
print("Response Text:", response.text)

Status Code: 200
Response Text: <!doctype html><html lang="en"><head><meta charset="utf-8"/><meta name="viewport" content="width=device-width,initial-scale=1,shrink-to-fit=no,user-scalable=no"/><meta name="theme-color" content="#FFFFFF"/><link id="favicon" rel="icon" href="/-/build/favicon.svg"/><link id="alternate-favicon" rel="alternate icon" href="/-/build/favicon.ico"/><link rel="mask-icon" href="/-/build/favicon_safari_mask.png" color="#FF2B2B"/><link rel="apple-touch-icon" href="/-/build/favicon_256.png"/><link rel="manifest" href="/-/build/manifest.json"/><script>!function(e,t,a,n,g){e[n]=e[n]||[],e[n].push({"gtm.start":(new Date).getTime(),event:"gtm.js"});var m=t.getElementsByTagName(a)[0],r=t.createElement(a);r.async=!0,r.src="https://www.googletagmanager.com/gtm.js?id=GTM-52GRQSL",m.parentNode.insertBefore(r,m)}(window,document,"script","dataLayer")</script><script>!function(){var e=window.analytics=window.analytics||[];if(!e.initialize)if(e.invoked)window.console&&console.e

- **1. Test de l'Application en Local**
*****
- Vérifions que toutes les fonctionnalités de l'application fonctionnent comme attendu.
Saisissons un SK_ID_CURR et assurons que la prédiction est retournée correctement.
*****
- **2. Test de l'Application en Ligne**
*****
- Accèdons à l'application via le lien fourni par Streamlit Cloud.
- Répètons les mêmes tests que pour l'application locale pour vérifier que tout fonctionne.
*****
- **3. Sauvegarde des Fichiers**
- Assurons-nous que les deux fichiers (app_local.py et app_online.py) sont bien organisés et documentés dans notre dépôt GitHub.